# Monoids to Monads

https://chjdev.com/2014/12/09/monoids-monads-python/


# Function Composition

In [ ]:
def f(x: int) -> int: return ~x
def g(x: int) -> int: return x ** 2
def h(x: int) -> int: return 2 * x

In [ ]:
f?

In [ ]:
x = 1
print(f(g(h(x))) == -5)
print(h(g(f(x))) == 8)

In [ ]:
def combine(x: '(int) -> int', y: '(int) -> int') -> '(int) -> int':
    return lambda _: x(y(_))

In [ ]:
combine?

In [ ]:
print(combine(f, combine(g, h))(x) == combine(combine(f, g), h)(x) == -5)

## Infix Operators

http://code.activestate.com/recipes/384122/

In [ ]:
# http://code.activestate.com/recipes/384122/

class Infix:
    def __init__(self, function):
        self.function = function
    def __ror__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __or__(self, other):
        return self.function(other)
    def __rlshift__(self, other):
        return Infix(lambda x, self=self, other=other: self.function(other, x))
    def __rshift__(self, other):
        return self.function(other)
    def __call__(self, value1, value2):
        return self.function(value1, value2)


In [ ]:
ø = Infix(combine)
print((f |ø| g |ø| h)(x) != (h |ø| g |ø| f)(x))
print((f |ø| g |ø| h)(x) == f(g(h(x))))

## The Trivial Monad

In [ ]:
class Trivial():
    def __init__(self, value):
        self._value = value

    def __call__(self, *args, **kwargs):
        return self._value

In [ ]:
def tf(x: int) -> Trivial: return Trivial(f(x))
def tg(x: int) -> Trivial: return Trivial(g(x))
def th(x: int) -> Trivial: return Trivial(h(x))

In [ ]:
def bind(what: Trivial, other: '(a) -> Trivial') -> Trivial:
    return other(what())

In [12]:
programA = (lambda a: bind(th(a),
              lambda b: bind(tg(b),
                lambda c: tf(c))))(x)

In [13]:
programA??

Signature:   programA(*args, **kwargs)
Type:        Trivial
String form: <__main__.Trivial object at 0x7f00cd1a3518>
Docstring:   <no docstring>


In [14]:
def _do_block(bind, *stmts):
    assert len(stmts) > 0
    if len(stmts) == 1:
        return lambda y: stmts[0](y)
    else:
        return lambda x: bind(stmts[0](x),
                  _do_block(bind, *stmts[1:]))

def trivial(*stmts): return _do_block(bind, *stmts)

programB = trivial(
    th,
    tg,
    tf)(x)

print(programA() == programB())

True


## A Variation of the Maybe Monad


In [16]:
!pip install singledispatch

azure-datalake-store 0.0.19 has requirement msrest~=0.4.5, but you'll have msrest 0.5.0 which is incompatible.


In [20]:
from singledispatch import singledispatch

In [21]:
class Maybe(): pass

class Just(Maybe):
    def __init__(self, value):
        self._value = value

    def __call__(self, *args, **kwargs):
        return self._value

class Nothing(Maybe): pass

@singledispatch
def bind(what: Just, other: 'def f(a) -> Trivial: pass') -> Maybe:
    return other(what())

@bind.register(Nothing)
def bind_nothing(what: Nothing, _) -> Nothing:
    return what

def maybe(*stmts): return _do_block(bind, *stmts)

In [22]:
bind

<function __main__.bind>

In [23]:
def mfun1(x: int) -> Maybe:
    print('mfun1', x)
    return Just(x) if (x % 2) == 0 else Nothing()

def mfun2(x: int) -> Maybe:
    print('mfun2', x)
    return Just(x) if (x % 3) == 0 else Nothing()

def mfun3(x: int) -> Maybe:
    print('mfun3', x)
    return Just(x) if (x % 4) == 0 else Nothing()

programC = maybe(
    mfun1,
    mfun2,
    mfun3)
print(programC(12)())
try:
    print(programC(11)())
except:
    print('as expected')


mfun1 12
mfun2 12
mfun3 12
12
mfun1 11
as expected
